In [41]:
import requests
import pandas as pd
import json
from dateutil import parser

In [3]:
API_KEY = "062fcaf65c7a4f1e56230ca193f72e9b-70d22f09f9d48c22fe72aabf5b1821e9"
ACCOUND_ID = "101-012-21494048-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [4]:
session = requests.Session()

In [5]:
# autorizziamo la richiesta
session.headers.update({
    'Authorization': 'Bearer ' + API_KEY,
    'Content-Type': 'application/json'
    })

In [6]:
params = dict(
    count = 10,
    granularity = "M1",
    price= "MBA"
)

In [7]:
url = f'{OANDA_URL}/accounts/{ACCOUND_ID}/instruments'

In [8]:
res = session.get(url, params=params, data= None, headers=None)

In [9]:
res.status_code

200

In [10]:
data = res.json()

In [11]:
instrument_list = data['instruments']


In [12]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate' ]

In [13]:
url = f'{OANDA_URL}/accounts/{ACCOUND_ID}/instruments?instruments=BTC_USD'
res = session.get(url, params=params, data=None, headers=None)


In [14]:
data = res.json()


In [15]:
instrument_list.append(data['instruments'][0])

In [16]:
instrument_dict = {}
for i in instrument_list:
    key = i['name']
    instrument_dict[key] = { k: i[k] for k in key_i}



In [17]:
with open('../data/instrument.json', 'w') as f:
    f.write(json.dumps(instrument_dict, indent=2))

In [23]:
def fetch_candles(pair_name, count= 10, granularity = 'H1'):
    url = f'{OANDA_URL}/instruments/{pair_name}/candles'
    params = dict(
        count = count,
        granularity = granularity,
        price= "MBA"
    )

    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    # controllo la risposta\
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']

    return response.status_code, data

In [60]:

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()

    price = ['mid', 'bid', 'ask'] # prezzo medio, prezzo di vendita, prezzo di acquisto

    ohlc = ['o', 'h', 'l', 'c'] # open, high, low, close (mid_o, mid_h, mid_l, mid_c,...)

    final_data = []


    for d in data:

        if d['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse( d['time'])
        new_dict['volume'] = d['volume']
        

        for p in price:
            for o in ohlc:
                new_dict[f'{p}_{o}'] = float(d[p][o])
        final_data.append(new_dict)


    df = pd.DataFrame(final_data)

    return df



In [68]:
def crea_data_file(pair_name, count = 10,granularity = 'H1'):
    status_code, data = fetch_candles(pair_name, count, granularity)
    if status_code != 200:
        print(f'Error: {status_code}', pair_name, data)
        return
    if len(data) == 0:
        print(f'No data: {pair_name}')
        return
    df = get_candles_df(data)
    # salvo i dati in un file
    df.to_pickle(f'../data/{pair_name}_{granularity}.pkl')
    print(f'{pair_name} {granularity} {df.shape[0]} candele, da: {df.time.min()} a {df.time.max()} !')
    

In [70]:
crea_data_file('BTC_USD', count = 10, granularity = 'H1')

EUR_USD H1 9 candele, da: 2022-05-22 21:00:00+00:00 a 2022-05-23 05:00:00+00:00 !


In [75]:
valute = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD', 'BTC']

In [76]:
for p1 in valute:
    for p2 in valute:
        pr = f'{p1}_{p2}'
        if pr in instrument_dict:
            for g in ['H1', 'H4']:
                crea_data_file(pr, count = 4001, granularity = g)

EUR_USD H1 4000 candele, da: 2021-09-30 10:00:00+00:00 a 2022-05-23 05:00:00+00:00 !
EUR_USD H4 4000 candele, da: 2019-10-25 13:00:00+00:00 a 2022-05-23 01:00:00+00:00 !
EUR_GBP H1 4000 candele, da: 2021-09-30 08:00:00+00:00 a 2022-05-23 06:00:00+00:00 !
EUR_GBP H4 4000 candele, da: 2019-10-25 13:00:00+00:00 a 2022-05-23 01:00:00+00:00 !
EUR_JPY H1 4000 candele, da: 2021-09-30 02:00:00+00:00 a 2022-05-23 06:00:00+00:00 !
EUR_JPY H4 4000 candele, da: 2019-10-25 05:00:00+00:00 a 2022-05-23 01:00:00+00:00 !
EUR_CHF H1 4000 candele, da: 2021-09-30 11:00:00+00:00 a 2022-05-23 06:00:00+00:00 !
EUR_CHF H4 4000 candele, da: 2019-10-25 13:00:00+00:00 a 2022-05-23 01:00:00+00:00 !
EUR_NZD H1 4000 candele, da: 2021-09-30 11:00:00+00:00 a 2022-05-23 06:00:00+00:00 !
EUR_NZD H4 4000 candele, da: 2019-10-25 13:00:00+00:00 a 2022-05-23 01:00:00+00:00 !
EUR_CAD H1 4000 candele, da: 2021-09-30 11:00:00+00:00 a 2022-05-23 06:00:00+00:00 !
EUR_CAD H4 4000 candele, da: 2019-10-25 13:00:00+00:00 a 2022-05-